# Module 3: Feature Selection

In this lab you will learn about **feature selection**, 
which reduces the dimensionality of data for the following reasons:

1. Reduces overfitting by removing noise introduced by some of the features.
2. Reduces training time, which allows you to experiment more with different models and hyperparameters.
3. Reduces data acquisition requirements.
4. Improves comprehensibility of the model because a smaller set of features is more comprehendible to humans. That will enable you to focus on the main sources of predictability, make the model more justifiable to another person.

For this session, we are going to use **red wine quality** dataset as a starting point for learning feature selection,
and then select the most relevant feature for predicting wine quality.



Feature selection methods generally falls into two categories known as **wrapper methods** and **filter methods**. There is a 3rd category, **embedded methods**, which includes those models where features are identified while learning the model parameters (e.g., LASSO). In this notebook we will discuss wrapper and filter methods.


Wrappers utilize the learning machine of interest as a black box to score subsets of
variable according to their predictive power. Filters select subsets of variables as a pre-processing
step, independently of the chosen predictor. Embedded methods perform v


<span style="text-decoration: underline;">**Filter methods**</span> apply a statistical measure and assign a score to each feature one at a time. These methods select features independent of predictive methods and run very fast. 
In this lab, you will go through **Pearson's χ²** and **ANOVA F-value based feature selection** in this section. 


<span style="text-decoration: underline;">**Wrapper methods**</span> use predictive methods for selecting a subset of features. The idea is to learn a set of features with feedbacks from the predictive method of interest. Based on the results drawn from the predictive model trained on that subset of features, 
they are either added or removed from the subset.
The problem is essentially reduced to a search problem.
[Greedy algorithms](https://en.wikipedia.org/wiki/Greedy_algorithm) 
are the most desirable in multivariate feature selection scenario because 1) 
wrapper methods are usually computationally very expensive; 2) 
greedy algorithms don't necessary provide the optimal solution,
which is good becuase it makes them less prone to overfitting.



sklearn API reference:

+ [sklearn.feature_selection.SelectKBest](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)
+ [sklearn.feature_selection.chi2](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html)
+ [sklearn.feature_selection.f_regression](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html)
+ [sklearn.feature_selection.mutual_info_classif](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html)
+ [sklearn.feature_selection.RFE](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)



In [ ]:
import os, sys
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import RFE

from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.base import clone

# Uncomment following line to view original output.
# np.random.seed(18937)

## Load Dataset

In [ ]:
# Dataset location
DATASET = '/dsa/data/all_datasets/wine-quality/winequality-red.csv'
assert os.path.exists(DATASET)

# Load and shuffle
dataset = pd.read_csv(DATASET, sep=';').sample(frac = 1).reset_index(drop=True)
dataset.describe()

Store features and labels into variables **X** and **y** respectively.

In [ ]:
X = dataset.iloc[:, :-1].to_numpy()
y = dataset.quality

## Feature selection solution space

From algorithm analysis point of view, a solution to feature selection problems can be respresented as a boolean vector,
each component indicating whether the corresponding feature has been selected.
For instance,

In [ ]:
selected = np.array([False, True, True, False, False, True, True, False, False, False, True])

sci-kit learn calls the corresponding indices to feature columns selected "support", 
which can be obtained using [np.flatnonzero()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.flatnonzero.html):

In [ ]:
support = np.flatnonzero(selected)
print(support)

Thus a naive approach that exhaustively search all subsets of features would have to verify $2^p$ solutions for $p$ features,
which translates into $\Omega(2^p)$ [time complexity](https://en.wikipedia.org/wiki/Time_complexity).
That is to say it would be very inefficient in practice.

However, we will run an exhaustive search for all solutions that provide 5 features to establish a baseline.
This limits time complexity to $O(p^5 \cdot n)$, assuming scoring a model takes linear time $O(n)$.
Therefore, the following cell checks all $\begin{pmatrix} 11 \\ 5 \end{pmatrix} = \frac{11\times10\times9\times8\times7}{5\times4\times3\times2\times1}=462$ solutions, 
and displays top 3 subset of features ranked by accuracy.

In Part 1 "Wrapper methods", we will use these solutions as comparison.

In [ ]:
import itertools

def search_combinations(estimator, X, y, k=5):
    # fit and score model based on some subset of features
    score = lambda X_features: clone(estimator).fit(X_features, y).score(X_features, y)
    
    # enumerate all combinations of 5 features
    for subset in itertools.combinations(range(X.shape[1]), 5):
        print(subset)
        sc = score(X[:, subset])
        print(sc)
        yield score(X[:, subset]), subset
        
sorted(search_combinations(LinearRegression(), X, y), reverse=True)

## Part 1: Wrapper methods

Within wrapper methods, there are different strategies for selecting the features. Here we will learn about **forward selection**, **backward elimination** and **recursive feature elimination** strategies.

### A. Forward selection

Forward selection is an iterative method in which we start with having no feature in the model. 
In each iteration, we keep adding the feature which best improves our model.

In [ ]:
def forward_select(estimator, X, y, k=5):
    # this array holds indicators of whether each feature is currently selected
    selected = np.zeros(X.shape[1]).astype(bool)
    
    # fit and score model based on some subset of features
    score = lambda X_features: clone(estimator).fit(X_features, y).score(X_features, y)  # accurary is the measure
    
    # find indices to selected columns
    selected_indices = lambda: list(np.flatnonzero(selected))
    
    # repeat till k features are selected
    while np.sum(selected) < k:
        # indices to unselected columns
        rest_indices = list(np.flatnonzero(~selected))
    
        # compute model scores with an additional feature
        scores = [score(X[:, selected_indices() + [i]]) for i in rest_indices]
        print('\n%accuracy if adding column:\n   ',
              {i:int(s*100) for i,s in zip(rest_indices,scores)})
        
        # find index within `rest_indices` that points to the most predictive feature not yet selected 
        idx_to_add = rest_indices[np.argmax(scores)]
        print('add column', idx_to_add)
        
        # select this new feature
        selected[idx_to_add] = True
        
    return selected_indices()

support = sorted(forward_select(LinearRegression(), X, y))
print(support)

### B. Backward elimination

In backward elimination, 
we start with all the features and remove the _least significant_ feature at each iteration,
which improves the performance of the model.

In [ ]:
def backward_eliminate(estimator, X, y, k=5):
    # this array holds indicators of whether each feature is currently selected
    selected = np.ones(X.shape[1]).astype(bool)
    
    # fit and score model based on some subset of features
    score = lambda X_features: clone(estimator).fit(X_features, y).score(X_features, y)
    
    # find indices to selected columns
    selected_indices = lambda: list(np.flatnonzero(selected))
    
    # repeat till k features are selected
    while np.sum(selected) > k:
        # Compute model scores with one of the features removed
        scores = [score(X[:, list(set(selected_indices()) - {i})]) for i in selected_indices()]
        print('\n%accuracy if removing column:\n   ',
              {i:int(s*100) for i,s in zip(selected_indices(), scores)})
        
        # Find index that points to the least predictive feature
        idx_to_remove = selected_indices()[np.argmax(scores)]
        print('remove column', idx_to_remove)
        
        # Remove this feature
        selected[idx_to_remove] = False
        
    return selected_indices()

support = sorted(backward_eliminate(LinearRegression(), X, y))
print(support)

### C. Recursive feature elimination

**Recursive Feature elimination** is an even more greedy algorithm provided by sklearn, 
which finds good performing feature subset with high efficiency. This method has similarity with backward elimination technique. 

The importance of each feature is obtained either through a **`coef_`** attribute 
or through a **`feature_importances_`** attribute.
So in order for recursive feature elimination algorithm in sklearn to work, 
the model is required to provide either of these attributes.

Usually, we start off using a low complexity model and use it as a benchmark for feature selection.

In [ ]:
model = LinearRegression()
selector = RFE(model, n_features_to_select=5)
selector.fit(X, y)
print("Num Features:", selector.n_features_)
print("Selected Features:", np.flatnonzero(selector.support_))

Then we can transform dataset to include only these features.

In [ ]:
X_new = selector.transform(X)
print(X_new.shape)

## Part 2: Filter methods

In filter methods, features are ranked in terms of predictiveness one by one, 
as opposed to considering a subset.
They incorporate statistical or information theoretic measures to rank each feature instead of measuring accuracy of a model trained on selected features.

### A. Pearson's χ² test based feature selection

The following cell shows the usage of a feature selector based on Pearson's χ² test.
This constructs the approximate χ² distribution and scores each feature vs 
the label in order to determine which feature is more relevant, 
one at a time, then selects features according to the score.

In [ ]:
selector = SelectKBest(chi2, k=5)
selector.fit(X, y)
print('χ² statistic', selector.scores_)
print('Selected indices', selector.get_support(True))

We can call the **transform()** method to select those feature columns from dataset and 
store into a new variable **X_selected**.

In [ ]:
X_selected = selector.transform(X)
X_selected.shape

**selector.transform()** does the same as slicing out these columns.

In [ ]:
np.allclose(X_selected, X[:, [1, 2, 5, 6, 10]])

Next we will take a closer look at this procedure by implementing one so we can better distinguish different feature selection methods. 
χ² test has many applications.
For feature selection, we utilize χ² statistic to test for dependence of each feature towards determining label.

**Note:** _You do NOT need to learn by heart the details of the computation to work on practices._

#### Step 1: Encode labels into orthogonal vector space

This is also know as **one-hot encoding**, which is applicable to classification problems.
Consider an example where you have defined 3 categories for possible outcomes: A, B and C.
In order for machine learning algorithms to be able to handle this type of data,
we have to convert them into numbers.
One-hot encoding uses a vector $ (y_1, y_2, y_3) $ where 
$$ y_i = \left[ \text{result falls into i}^{th}\text{ category} \right] \in \left\{ 0, 1 \right\} $$
Therefore, one-hot encoding for A, B and C categories becomes (1, 0, 0), (0, 1, 0) and (0, 0, 1) respectively.
This has an advantage over plainly translating A, B and C into 1, 2 and 3 (a.k.a **sparse encoding**)
in a way that orthogonal vectors do not impose assumptions of their order or magnitudes between categories like numbers would.

For example, 3>1 is true, how ever it doesn't mean to imply C>A or C is superior to A in any way.
However this would affect the model's numerical stability.

Therefore one-hot encoding is an widely adopted technique for processing categories. 
Sparse encoding could be used when persisting a dataset in order to save storage space.

**Note:** If you recall _regression in R_ from 8610 (Stat/Math), 
one-hot encoding is how the categorical / nominal variables are encoded as 
independent predictors in the regression formula.


In [ ]:
from sklearn.preprocessing import LabelBinarizer
Y = np.array(LabelBinarizer().fit_transform(y))
print(Y.shape)
print(Y[:5])

#### Step 2: Compute the [contingency table](https://en.wikipedia.org/wiki/Contingency_table) of observed frequencies

**observed** is a (#classes)-by-(#features) matrix that contains the "number of occurrences" for each combination of feature and classes.

**Note:** You previously saw contingency tables in the 8610 (Stat/Math) class and computed the Chi-Squared (χ²) statistic there.

In [ ]:
observed = np.dot(Y.T, X)

#### Step 3: Compute the expected frequencies using marginal frequencies

**expected** has the same shape as **observed** matrix, but represent the expected frequencies in theory.

In [ ]:
expected = np.dot(
    Y.mean(axis=0).reshape(1, -1).T, # Mean value for all classes (transposed)
    X.sum(axis=0).reshape(1, -1) # Marginal frequencies for all features
)

#### Step 4: Compute the χ² statistic between **observed** and **expected**

In [ ]:
chi_squared = np.sum((observed-expected)**2 / expected, axis=0)
print(chi_squared)

Compare with sklearn **chi2()**, which returns two arrays containing χ² statistic and p-value, respectively.

In [ ]:
chi2_sklearn, pvalue_sklearn = chi2(X, y)
print("Chi-Squared Statistic")
print(chi2_sklearn)

print("P-Values")
print(pvalue_sklearn)

#### Step 5: Rank features descendingly by χ² statistic

Optionally, then sort these indices so they remain relative order.

In [ ]:
support = sorted(np.argsort(-chi_squared)[:5])
print(support)

#### Step 6: Select these features and transform dataset

In [ ]:
X_new = X[:, support]
np.allclose(X_new, X_selected)

### A note on the p-value

sklearn not only has provided the **χ² statistic** but also **p-value**,
which is very useful because p-value could tell you quantitively how probable each feature is relevant,
which in turn helps you decide how many and which features are worthwhile to retain.

Here's part of a **χ² distribution** table.
In our dataset, we have (#classes - 1) = 5 degrees of freedom (d.o.f.).

<table cellspacing="2" cellpadding="3" border="1" align="center">
<tbody>
<tr class="col1"><td bgcolor="#009bff">df</td><td bgcolor="#009bff">0.995</td><td bgcolor="#009bff">0.99</td><td bgcolor="#009bff">0.975</td><td bgcolor="#009bff">0.95</td><td bgcolor="#009bff">0.9</td><td bgcolor="#009bff">0.1</td><td bgcolor="#009bff">0.05</td><td bgcolor="#009bff">0.025</td><td bgcolor="#009bff">0.01</td><td bgcolor="#009bff">0.005</td></tr>
<tr class="col1"><td bgcolor="#009bff">1</td><td bgcolor="#cdebfa">0</td><td bgcolor="#edfbfa">0</td><td bgcolor="#cdebfa">0.001</td><td bgcolor="#edfbfa">0.004</td><td bgcolor="#cdebfa">0.016</td><td bgcolor="#edfbfa">2.706</td><td bgcolor="#cdebfa">3.841</td><td bgcolor="#edfbfa">5.024</td><td bgcolor="#cdebfa">6.635</td><td bgcolor="#edfbfa">7.879</td></tr>
<tr class="col1"><td bgcolor="#009bff">2</td><td bgcolor="#edfbfa">0.01</td><td bgcolor="#cdebfa">0.02</td><td bgcolor="#edfbfa">0.051</td><td bgcolor="#cdebfa">0.103</td><td bgcolor="#edfbfa">0.211</td><td bgcolor="#cdebfa">4.605</td><td bgcolor="#edfbfa">5.991</td><td bgcolor="#cdebfa">7.378</td><td bgcolor="#edfbfa">9.21</td><td bgcolor="#cdebfa">10.597</td></tr>
<tr class="col1"><td bgcolor="#009bff">3</td><td bgcolor="#cdebfa">0.072</td><td bgcolor="#edfbfa">0.115</td><td bgcolor="#cdebfa">0.216</td><td bgcolor="#edfbfa">0.352</td><td bgcolor="#cdebfa">0.584</td><td bgcolor="#edfbfa">6.251</td><td bgcolor="#cdebfa">7.815</td><td bgcolor="#edfbfa">9.348</td><td bgcolor="#cdebfa">11.345</td><td bgcolor="#edfbfa">12.838</td></tr>
<tr class="col1"><td bgcolor="#009bff">4</td><td bgcolor="#edfbfa">0.207</td><td bgcolor="#cdebfa">0.297</td><td bgcolor="#edfbfa">0.484</td><td bgcolor="#cdebfa">0.711</td><td bgcolor="#edfbfa">1.064</td><td bgcolor="#cdebfa">7.779</td><td bgcolor="#edfbfa">9.488</td><td bgcolor="#cdebfa">11.143</td><td bgcolor="#edfbfa">13.277</td><td bgcolor="#cdebfa">14.86</td></tr>
<tr class="col1"><td bgcolor="#009bff">5</td><td bgcolor="#cdebfa">0.412</td><td bgcolor="#edfbfa">0.554</td><td bgcolor="#cdebfa">0.831</td><td bgcolor="#edfbfa">1.145</td><td bgcolor="#cdebfa">1.61</td><td bgcolor="#edfbfa">9.236</td><td bgcolor="#cdebfa">11.07</td><td bgcolor="#edfbfa">12.833</td><td bgcolor="#cdebfa">15.086</td><td bgcolor="#edfbfa">16.75</td></tr>
<tr class="col1"><td bgcolor="#009bff">6</td><td bgcolor="#edfbfa">0.676</td><td bgcolor="#cdebfa">0.872</td><td bgcolor="#edfbfa">1.237</td><td bgcolor="#cdebfa">1.635</td><td bgcolor="#edfbfa">2.204</td><td bgcolor="#cdebfa">10.645</td><td bgcolor="#edfbfa">12.592</td><td bgcolor="#cdebfa">14.449</td><td bgcolor="#edfbfa">16.812</td><td bgcolor="#cdebfa">18.548</td></tr>
<tr class="col1"><td bgcolor="#009bff">7</td><td bgcolor="#cdebfa">0.989</td><td bgcolor="#edfbfa">1.239</td><td bgcolor="#cdebfa">1.69</td><td bgcolor="#edfbfa">2.167</td><td bgcolor="#cdebfa">2.833</td><td bgcolor="#edfbfa">12.017</td><td bgcolor="#cdebfa">14.067</td><td bgcolor="#edfbfa">16.013</td><td bgcolor="#cdebfa">18.475</td><td bgcolor="#edfbfa">20.278</td></tr>
<tr class="col1"><td bgcolor="#009bff">8</td><td bgcolor="#edfbfa">1.344</td><td bgcolor="#cdebfa">1.646</td><td bgcolor="#edfbfa">2.18</td><td bgcolor="#cdebfa">2.733</td><td bgcolor="#edfbfa">3.49</td><td bgcolor="#cdebfa">13.362</td><td bgcolor="#edfbfa">15.507</td><td bgcolor="#cdebfa">17.535</td><td bgcolor="#edfbfa">20.09</td><td bgcolor="#cdebfa">21.955</td></tr>
<tr class="col1"><td bgcolor="#009bff">9</td><td bgcolor="#cdebfa">1.735</td><td bgcolor="#edfbfa">2.088</td><td bgcolor="#cdebfa">2.7</td><td bgcolor="#edfbfa">3.325</td><td bgcolor="#cdebfa">4.168</td><td bgcolor="#edfbfa">14.684</td><td bgcolor="#cdebfa">16.919</td><td bgcolor="#edfbfa">19.023</td><td bgcolor="#cdebfa">21.666</td><td bgcolor="#edfbfa">23.589</td></tr>
<tr class="col1"><td bgcolor="#009bff">10</td><td bgcolor="#edfbfa">2.156</td><td bgcolor="#cdebfa">2.558</td><td bgcolor="#edfbfa">3.247</td><td bgcolor="#cdebfa">3.94</td><td bgcolor="#edfbfa">4.865</td><td bgcolor="#cdebfa">15.989</td><td bgcolor="#edfbfa">18.307</td><td bgcolor="#cdebfa">20.483</td><td bgcolor="#edfbfa">23.209</td><td bgcolor="#cdebfa">25.188</td></tr>
</tbody>
</table>



Our null hypothesis is that a class labels is independent of a feature. 
Since here we used **χ² statistic** for testing what the null hypothesis has claimed : **independence between features and labels**.
We can choose `p-value` $>2.5\%$ as the critical level at which the null hypothesis **can not be rejected**,
and reject the hypothesis otherwise.
This is effectively saying that, if the feature is more that 10% chance to be independent of the class, then it is not a good predictor.
The alternative hypothesis is, therefore, that the feature is probable predictor of the class, i.e., the class is dependent on the feature.
The following table would summarize our conclusion based on this criterion:

<table>
<tr><td><strong>feature idx</strong></td><td>0</td><td>1</td><td>2</td><td>3</td><td>4</td><td>5</td><td>6</td><td>7</td><td>8</td><td>9</td><td>10</td></tr>
<tr><td><strong>χ²</strong></td><td>11.3</td><td>15.6</td><td>13.0</td><td>4.12</td><td>0.752</td><td>162</td><td>2.76e+03</td><td>2.30e-04</td><td>0.155</td><td>4.56</td><td>46.4</td></tr>
<tr><td><strong>p-value</strong></td><td>~5%</td><td>~0.5%</td><td>~1%</td><td>90%~95%</td><td>100%</td><td>0%</td><td>0%</td><td>100%</td><td>100%</td><td>90%~95%</td><td>0%</td></tr>
<tr><td><strong>interpretation</strong></td><td>independent</td><td><strong>dependent</strong></td><td><strong>dependent</strong></td><td>independent</td><td>independent</td><td><strong>dependent</strong></td><td><strong>dependent</strong></td><td>independent</td><td>independent</td><td>independent</td><td><strong>dependent</strong></td></tr>
</table>

So we should select feature 1, 2, 5, 6 and 10.

<!-- future work:
sklearn learn can help make this process more precise and easy than looking up the **χ² distribution** table.
Following cells prints the all the p-values:
-->

Please review the documentation of the chi2 function from `sklearn.feature_selection`.

In [ ]:
help(chi2)


--- 

Here's how to generate χ² table, which may come in handy now and then.

In [ ]:
def chi2_table(degree_of_freedoms):
    from scipy.stats import chi2 as chi2_distribution
    pvalue = np.array([0.995, 0.99, 0.975, 0.95, 0.90, 0.10, 0.05, 0.025, 0.01, 0.005])
    return pd.DataFrame(chi2_distribution.isf(
            # isf(p) = inverse(1-cdf)(p) which takes p-value returns chi square value
            #     where cdf is short for cumulative distribution function
        pvalue, np.expand_dims(degree_of_freedoms, 1)),
        columns = pvalue, index = degree_of_freedoms)

chi2_table(range(5, 12))

--- 
### B. ANOVA F-value based feature selection

The following cell shows the usage of a feature selector based on ANOVA F-value and Pearson's correlation.
This calculates Pearson's correlation of each feature vs the label in order to determine which feature is more relevant, 
one at a time, then selects features according to the ANOVA F-value derived from Pearson's correlation.

See also: https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.pearsonr.html

**Note:** You previously saw the ANOVA and MANOVA within the 8610 (Stat/Math) course.

In [ ]:
selector = SelectKBest(f_regression, k=5)
selector.fit(X, y)
print('score', selector.scores_)
print('Selected indices', selector.get_support(True))

Next we will take a closer look at this procedure to gain a more solid understanding.

**Note:** _You do NOT need to learn by heart the details of the computation to work on practices._

#### Step 1: Compute cross correlation

$$r_j = \frac{\sigma_{X_j y}}{\sigma_{X_j} \sigma_y} = \frac{(y-\bar y)^T (X_j-\bar {X_j})}{\lVert X_j-\bar {X_j}\rVert \cdot \lVert y-\bar y\rVert}$$


In [ ]:
from sklearn.preprocessing import scale
X_centered = scale(X.astype('float'), with_std = False)
y_centered = scale(y.astype('float'), with_std = False)
corr = np.dot(y_centered, X_centered) / np.linalg.norm(X_centered, axis = 0) / np.linalg.norm(y_centered)
print(corr)

In [ ]:
from scipy.stats import pearsonr
print([pearsonr(X[:,i], y)[0] for i in range(X.shape[1])])

#### Step 2: Compute ANOVA F-value

This is a F-test for correlation coefficients.
Read more [here](https://onlinecourses.science.psu.edu/stat501/lesson/2/2.6).
Similar to the way that χ² test is provided by sklearn, [f_regression()](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression) also supplied p-value for quantitatively assessing how relevant each feature is.

$$ F = t^2 = \left( \frac{r\sqrt{n-2}}{\sqrt{1-r^2}} \right) ^2 = \frac{r^2/1 }{(1-r^2)/(n-2)} $$

In [ ]:
corr2 = corr ** 2
Fvalue = corr2 / (1 - corr2) * (y.shape[0] - 2)
print(Fvalue)

Compare to f_regression().

In [ ]:
Fvalue_sklearn, pvalue2_sklearn = f_regression(X,y)
print(Fvalue_sklearn)

#### Step 3: Select these features and transform dataset

In [ ]:
support = sorted(np.argsort(-Fvalue)[:5])
print(support)

### More on F-value and p-value

F-test can be used as hypothesis testing for a ratio of two χ² distributions.
The F-test for correlation coefficients is derived from testing a ratio between regression sum square (SSR)
and error sum square (SSE).

$$ SSR = \sum _{i=1}^n {\left(\hat {y_i} - \bar y \right)^2}$$

$$ SSE = \sum _{i=1}^n {\left(\hat {y_i} - y_i \right)^2}$$

$$ F  = \frac{SSR/(v-1) }{SSE/(n-2)} $$

With correlation coefficients, this is 

$$ F = \frac{r^2/1 }{(1-r^2)/(n-2)} $$

with degree of freedom 1 and (n-2) respectively.

Its null hypothesis claims r = 0, i.e. the independence of each feature and label.
We could theoretically make hypothesis testing using an F-distribution table.

Here's how to generate F table.

In [ ]:
def f_table(alpha, v1, v2):
    from scipy.stats import f as f_distribution
    v1 = np.array(list(v1)); v2 = np.array(list(v2))
    return pd.DataFrame(f_distribution.isf(alpha, v1[np.newaxis, ...], v2[..., np.newaxis]),
        columns = v1, index = v2)

import itertools
f_table(0.1, range(1, 9), itertools.chain(range(1,10), range(10, 260, 20)))

Alternatively, directly compute p-value.

In [ ]:
from scipy.stats import f as f_distribution
f_distribution.sf(Fvalue, 1, y.shape[0] - 2)

Compare to those obtained from f_regression().

In [ ]:
pvalue2_sklearn

Print in percentages perhaps makes it easier to read.

In [ ]:
np.round(pvalue2_sklearn * 100)

### Recap

We just went through **χ²** and **F-value** based feature selection in detail because 
these two different methods work better on classification and regression respectively.
The general steps for doing feature selection with sklearn was:

1. Choose a feature scoring method
2. Initialize a feature selector
3. Fit feature selector on the data

Other feature selection methods provided by sklearn include:

* Classification: chi2, f_classif, mutual_info_classif
* Regresssion: f_regression, mutual_info_regression



# Conclusion

In this lab, we learned about:

* Wrapper methods
    * Forward selection
    * Backward elimination
    * Recursive feature elimination
    
 
* Filter methods
    * Pearson's χ²
    * ANOVA F-value


And once again,
1. Wrapper methods are usually computationally expensive
2. Greedy algorithms don't necessary provide the optimal solution, which may be good becuase it makes them less prone to overfitting.